In [2]:
from model import Model

models = [Model(f'models/{name}.hdf5')
          for name in ['model-5-20-0.591', 'model-10-20-0.537', 'model-20-20-0.478']]

In [3]:
import pandas as pd

df = pd.read_csv('../data/full/movies_edited.csv')
df


,movieId,title,genres,movie_idx
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,2.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4.0
4,5,Father of the Bride Part II (1995),Comedy,5.0
...,...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed),37535.0
58094,193878,Les tribulations d'une caissière (2011),Comedy,37536.0
58095,193880,Her Name Was Mumu (2016),Drama,37537.0
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi,37538.0


In [11]:
import math
import json

count_genres = {}
count_features = {}

for modelIndex, model in enumerate(models):
  for i, row in df.iterrows():
    if not math.isnan(row.movie_idx):
      id = int(row.movie_idx)
      features = model.embeddings[id]
      # genres
      for genre in row.genres.split('|'):   
        if genre not in count_genres:
          count_genres[genre] = {
            'count': 0,
            'features': {}
          }
        for i in range(len(features)):
          if i not in count_genres[genre]['features']:
            count_genres[genre]['features'][i] = 0
          count_genres[genre]['features'][i] += features[i]
        count_genres[genre]['count'] += 1
      # features
      for i in range(len(features)):
        for genre in row.genres.split('|'):   
          if i not in count_features:
            count_features[i] = {}
          if genre not in count_features[i]:
            count_features[i][genre] = 0
          count_features[i][genre] += features[i]

  with open(f'info/genres-{modelIndex}.json', 'w') as f:
    json.dump(count_genres, f)

  with open(f'info/features-{modelIndex}.json', 'w') as f:
    json.dump(count_features, f)